In [7]:
import os

from openpyxl import load_workbook


base_directory = os.getcwd()
datasets_directory = os.path.join(base_directory, "datasets/")


data = {}
for year_dir in os.scandir(datasets_directory):
    if year_dir.is_dir():
        data[year_dir.name] = {}
        for file in os.scandir(year_dir.path):
            if file.is_file():
                payments = []
                wb = load_workbook(filename = file.path)
                ws = wb['Data']
                for row in ws.rows:
                    payments.append((row[1].value, float(row[2].value)))
                
                data[year_dir.name][file.name.split('.')[0]] = payments
                wb.close()

/home/hokage/.conda/envs/mathelp/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
weights_file = open("datasets/weights.pickle", 'wb')
weights = {}

from openpyxl import load_workbook
import pickle

wb = load_workbook(filename="./datasets/weights.xlsx")
ws = wb['Sheet1']
for row in ws.rows:
    if row[0].value is not None:
        try:
            weights[str(row[0].value)] = float(str(row[1].value or ''))
        except ValueError:
            print("Ehllo")
wb.close()
pickle.dump(weights, weights_file)
weights_file.close()

Ehllo


In [9]:
import pickle

dataset = []

for category in weights:
    for year in data:
        for month in data[year]:
            budget = sum([payment[1] for payment in data[year][month]])
            count = len(list((payment for payment in data[year][month] if payment[0] == category)))
            if count> 0:
                dataset.append(
                    (
                        category, 
                        year, 
                        month, 
                        sum([float(payment[1]) for payment in data[year][month] if payment[0] == category]),
                        sum([float(payment[1]) for payment in data[year][month] if payment[0] == category]) / float(budget),
                        count
                    )
                )

datacases_file = open("./datasets/datasets.pickle", 'wb')
pickle.dump(dataset, datacases_file)
datacases_file.close()
